In [53]:
import pandas as pd
import numpy as np
contest=pd.read_csv('WPX_CW_2023.csv')

In [69]:
# Easy Markdown formatting
def df_to_md(pd):
    text=pd.to_markdown();
    for a in text.split('\n'):
        print(f"{a}")

from datetime import timedelta, datetime

def ceil_date(date, **kwargs):
    secs = timedelta(**kwargs).total_seconds()
    return datetime.fromtimestamp(date.timestamp() + secs - date.timestamp() % secs)

def floor_date(date, **kwargs):
    secs = timedelta(**kwargs).total_seconds()
    return datetime.fromtimestamp(date.timestamp() - date.timestamp() % secs)

contest['date']=pd.to_datetime(contest.date,format='%Y-%m-%d %H:%M:%S',errors="ignore")     
contest['date_rounded_5m'] =contest['date'].apply(lambda x: floor_date(x, minutes=5))      
contest=contest.dropna()

print(f"Total records {len(contest)}")

Total records 3900762


In [70]:
# Lets just see the 10M band
TenM = contest[contest.band=='10m']
DU_Spots_All = contest[contest.dx_pfx=='DU']
DU_Spots_10M = contest[(contest.dx_pfx=='DU')&(contest.band=='10m')]
print(f"We have {len(DU_Spots_10M)} spots for DU calls on 10m band.")


We have 3417 spots for DU calls on 10m band.


In [71]:
# Lets Summarize 10M DU Spots 
du_10m_summary=DU_Spots_10M.groupby(['dx'])['callsign'].count().sort_values(ascending=False).reset_index()
du_10m_summary.columns=['Call','Spotted_Count']

In [72]:
df_to_md(du_10m_summary[du_10m_summary.Spotted_Count>10])

|    | Call   |   Spotted_Count |
|---:|:-------|----------------:|
|  0 | 4F3OM  |            1009 |
|  1 | 4I1EBC |             789 |
|  2 | DU3T   |             500 |
|  3 | DV3A   |             488 |
|  4 | DU1WBX |             256 |
|  5 | 4D3X   |             188 |
|  6 | 4F3BZ  |              99 |
|  7 | DU1EV  |              31 |
|  8 | DU1VGX |              16 |
|  9 | DX9EVM |              13 |


In [77]:
DU_QRP = contest[(contest.dx_pfx=='DU')&(contest.band=='10m') & ((contest.dx=='4E1ABC') |(contest.dx=='4F3OM'))]
DU_HP = contest[(contest.dx_pfx=='DU')&(contest.band=='10m') & ((contest.dx=='DU3T') |(contest.dx=='DV3A')|(contest.dx=='DU1EV'))]
DU_LP=contest[(contest.dx_pfx=='DU')&(contest.band=='10m') & ((contest.dx=='4D3X') |(contest.dx=='4E1AGW')|(contest.dx=='4F3BZ'))]

In [87]:
HP_vs_QRP=pd.merge(DU_HP,DU_QRP,on=['callsign','date_rounded_5m'],how='inner')[['date_rounded_5m','callsign','freq_x','dx_x','freq_y','dx_y','db_x','db_y']]
df_to_md(HP_vs_QRP[HP_vs_QRP.db_y >= HP_vs_QRP.db_x])

|     | date_rounded_5m     | callsign   |   freq_x | dx_x   |   freq_y | dx_y   |   db_x |   db_y |
|----:|:--------------------|:-----------|---------:|:-------|---------:|:-------|-------:|-------:|
|  16 | 2022-05-28 08:50:00 | VK6ANC     |  28037.6 | DV3A   |  28019.9 | 4F3OM  |     22 |     25 |
|  36 | 2022-05-28 10:30:00 | VU3KAZ     |  28024.4 | DU3T   |  28032   | 4F3OM  |     21 |     21 |
|  48 | 2022-05-28 10:50:00 | BG4WOM     |  28024.7 | DU3T   |  28032.2 | 4F3OM  |     25 |     31 |
|  49 | 2022-05-28 10:50:00 | BG4WOM     |  28053.5 | DV3A   |  28032.2 | 4F3OM  |     24 |     31 |
|  61 | 2022-05-28 11:00:00 | BD7LAE     |  28024.4 | DU3T   |  28039.9 | 4F3OM  |     32 |     32 |
| 141 | 2022-05-30 06:30:00 | VK4CT      |  28002.1 | DV3A   |  28032.9 | 4F3OM  |     13 |     14 |


In [90]:
LP_vs_QRP=pd.merge(DU_LP,DU_QRP,on=['callsign','date_rounded_5m'],how='inner')[['date_rounded_5m','callsign','freq_x','dx_x','freq_y','dx_y','db_x','db_y']]
df_to_md(LP_vs_QRP[LP_vs_QRP.db_y >= LP_vs_QRP.db_x])

|    | date_rounded_5m     | callsign   |   freq_x | dx_x   |   freq_y | dx_y   |   db_x |   db_y |
|---:|:--------------------|:-----------|---------:|:-------|---------:|:-------|-------:|-------:|
| 30 | 2022-05-29 14:00:00 | JH7CSU1    |  28009.2 | 4F3BZ  |  28040.1 | 4F3OM  |     12 |     21 |
| 34 | 2022-05-29 14:30:00 | JA4ZRK     |  28033   | 4F3BZ  |  28001   | 4F3OM  |      3 |      8 |
| 46 | 2022-05-29 16:05:00 | JH7CSU1    |  28034.1 | 4F3BZ  |  28023.6 | 4F3OM  |     18 |     20 |
| 47 | 2022-05-29 16:10:00 | VU3KAZ     |  28034   | 4F3BZ  |  28023.5 | 4F3OM  |     15 |     17 |


,callsign,de_pfx,de_cont,freq,band,dx,dx_pfx,dx_cont,mode,db,date,speed,tx_mode,date_rounded_5m
1025,DU3TW,DU,OC,21040.5,15m,JA6WFM,JA,AS,CQ,8.0,2022-05-28 00:00:12,30.0,CW,2022-05-28 08:00:00
1027,DU3TW,DU,OC,14023.9,20m,JA3YBK,JA,AS,CQ,25.0,2022-05-28 00:00:13,28.0,CW,2022-05-28 08:00:00
1028,DU3TW,DU,OC,14033.6,20m,IR2Q,I,EU,CQ,20.0,2022-05-28 00:00:13,37.0,CW,2022-05-28 08:00:00
1029,DU3TW,DU,OC,21017.9,15m,JR3RIU,JA,AS,CQ,18.0,2022-05-28 00:00:13,29.0,CW,2022-05-28 08:00:00
2835,DU3TW,DU,OC,21035.3,15m,KH6TU,KH6,OC,CQ,8.0,2022-05-28 00:00:23,31.0,CW,2022-05-28 08:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6918614,DU3TW,DU,OC,21025.0,15m,DS4EOI,HL,AS,CQ,4.0,2022-05-29 22:57:38,31.0,CW,2022-05-30 06:55:00
6921686,DU3TW,DU,OC,21032.4,15m,8J6YAB,JA,AS,CQ,13.0,2022-05-29 22:58:49,27.0,CW,2022-05-30 06:55:00
6926450,DU3TW,DU,OC,14046.0,20m,JA3YBK,JA,AS,CQ,5.0,2022-05-29 23:00:41,28.0,CW,2022-05-30 07:00:00
6928186,DU3TW,DU,OC,21016.0,15m,HL2BQG,HL,AS,CQ,5.0,2022-05-29 23:01:22,30.0,CW,2022-05-30 07:00:00
